In [88]:
import pandas as pd
import numpy as np
import math
from os import listdir
from os.path import isfile, join
from ipynb.fs.defs.functions import new_route

dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
onlyfiles = [f for f in listdir("../Dati/turni_f/") if isfile(join("../Dati/turni_f/", f))]

In [89]:
def get_table_cost(path, dframe):
    cols = ["lunedi","martedi","mercoledi","giovedi","venerdi","tot"]
    resume_table = pd.DataFrame(columns = cols)
    
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

    for file in onlyfiles:
        resume_table = pd.concat([resume_table, pd.Series(index=[file], dtype=float)])[cols]
        current_file = pd.read_csv(path+file, delimiter=";", encoding = "ISO-8859-1")
        for day in current_file.columns:
            resume_table[day].loc[file] = sum(new_route(current_file[day].dropna(), dframe).values)
        resume_table["tot"].loc[file] = sum(resume_table.loc[file].dropna())
    return resume_table

In [90]:
resume_time_table = get_table_cost("../Dati/turni_f/original_start/", dft)
resume_distance_table = get_table_cost("../Dati/turni_f/original_start/", dfd)

In [91]:
total_table = pd.DataFrame(columns=["time", "distance"])
col = ["time", "distance"]
for r in range(0, len(resume_time_table), 2):
    file = resume_time_table.index[r]
    curr_indx = file.replace('1', '')
    total_table = pd.concat([total_table, pd.Series(index=[curr_indx], dtype=float)])[col]
    total_table["time"].loc[curr_indx] = resume_time_table["tot"].iloc[r] + resume_time_table["tot"].iloc[r+1]
    total_table["distance"].loc[curr_indx] = resume_distance_table["tot"].iloc[r] + resume_distance_table["tot"].iloc[r+1]
total_table = total_table[col]
total_table

,time,distance
f_original.csv,1777,1691
original_2opt.csv,1434,1309
original_ls.csv,1568,1465
original_MVLS.csv,1399,1218


In [92]:
resume_time_table.to_csv("../Dati/altro/original_time_table.csv", sep=";", encoding = "ISO-8859-1")
resume_distance_table.to_csv("../Dati/altro/original_distance_table.csv", sep=";", encoding = "ISO-8859-1")
total_table.to_csv("../Dati/altro/original_total_table.csv", sep=";", encoding = "ISO-8859-1")